## Experiments with _Water meters_!
In this notebook we will train ResNet101, DenseNet121 and VGG19 to predict numbers on water meter.

In [1]:
import sys
import os

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqn
%matplotlib inline

sys.path.append('..')

import meters as ms
from meters.batch import MeterBatch
from meters.dataset import B,V,F, FilesIndex, Dataset, Pipeline
from meters.dataset.dataset.models.tf import ResNet101, DenseNet121, VGG19


plt.style.use('seaborn-poster')
plt.style.use('ggplot')

Create fileindex witch allows us to loading only part of images instead all images.

`src` - the path to images in blosc format.

In [2]:
src = './../shows/data/images/'

fileindex = FilesIndex(path=src+'*.blosc', no_ext=True)

dset = Dataset(fileindex, batch_class=MeterBatch)
dset.cv_split()

Write pipeline to load and crop data and labels

In [4]:
load_ppl = (
    Pipeline()
    .load(src=src, fmt='blosc', components='images')
    .load(src='./../shows/data/new_data.csv',\
          fmt='csv',\
          components=['coordinates', 'labels'],\
          index_col='file_name')
    .crop_from_bbox()
    .split_labels()
    .split_to_digits(n_digits=8)
    .resize(shape=(64, 32))
)

In [5]:
# добавить к названиям всех файлов и лейблам буковку
# обновить ноутбук
# дописать аппенд в ноутбук

In [6]:
a = (load_ppl<< dset.train).next_batch(1)

With config create model parameters 

In [6]:
inputs_config = {
    'images': {'shape': (64, 32, 3)},
    'labels': {'classes': (10), 'transform': 'ohe', 'name': 'targets'}
}

config = {
    'inputs': inputs_config,
    'optimizer': 'Adam',
    'loss': 'ce',
    'input_block/inputs': 'images',
    'head/units': 10,
    'output': dict(ops=['labels', 'proba', 'accuracy'])
}

We will experements with models. For this create dict with:
* `key` - model
* `value` - model name

In [7]:
models = [DenseNet121, VGG19, ResNet101]
models_names = ['DenseNet121', 'VGG19', 'ResNet101']
models_dict = dict(zip(models_names, models))

In [9]:
def get_ppl(model, name):
    """"Create train and test pipeline using given `model` and `name`
    
    Parameters
    ----------
    model : Dataset model
        preloaded model from dataset
    
    name : str
        name of model
    
    Returns
    -------
    train_ppl : Dataset pipeline
        train pipeline

    test_ppl : Dataset pipeline
        test pipeline"""
    train_ppl = (
        (load_ppl << dset.train)
        .init_variable('model', model)
        .init_variable('loss', init_on_each_run=list)
        .init_model('dynamic',
                    V('model'),
                    name,
                    config=config)
        .train_model(name,
                     fetches='loss',
                     feed_dict={'images': B('images'),
                                'labels': B('labels')},
                     save_to=V('loss'), 
                     mode='a')
    )

    test_ppl = (
        (load_ppl << dset.test)
        .init_variable('proba', init_on_each_run=list)
        .init_variable('target', init_on_each_run=list)
        .init_variable('accuracy', init_on_each_run=list)
        .import_model(name, train_ppl)
        .predict_model(name,
                       fetches=['output_proba',
                                'targets',
                                'output_accuracy'],
                       feed_dict={'images': B('images'),
                                  'labels': B('labels')},
                       save_to=[V('proba'), 
                                V('target'), 
                                V('accuracy')], 
                       mode='a')
    )
    pred_ppl = (
    (load_ppl << dset.test)
    .init_variable('accuracy', init_on_each_run=list)
    .import_model(name, train_ppl)
    .predict_model(name,
                   fetches=['output_accuracy'],
                   feed_dict={'images': B('images'),
                              'labels': B('labels')},
                   save_to=[V('accuracy')], 
                   mode='a')
    )
    return train_ppl, test_ppl

train pipelines with different models and save it after training

In [10]:
global_acc = []
for name, model in models_dict.items():
    print(name)
    train_ppl, test_ppl = get_ppl(model, name)
    for i in tqn(range(900)):
        tr = train_ppl.next_batch(25, n_epochs=None, suffle=True, drop_last=True)
        te = test_ppl.next_batch(25, n_epochs=None, suffle=True, drop_last=True)
    plt.plot(test_ppl.get_variable('accuracy'), label=name+'accuracy')
    plt.legend(fontsize=14)
    plt.show()
    global_acc.append(test_ppl.get_variable('accuracy'))
    train_ppl.save_model(name, path=name)

DenseNet121


/usr/local/lib/python3.5/dist-packages/scipy/ndimage/interpolation.py:616: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


ResourceExhaustedError: OOM when allocating tensor of shape [128] and type float
	 [[Node: DenseNet121/DenseNet121/body/block-3/conv-17/layer-0/batch_normalization/gamma/Adam/Initializer/zeros = Const[_class=["loc:@DenseNet121/body/block-3/conv-17/layer-0/batch_normalization/gamma"], dtype=DT_FLOAT, value=Tensor<type: float shape: [128] values: 0 0 0...>, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'DenseNet121/DenseNet121/body/block-3/conv-17/layer-0/batch_normalization/gamma/Adam/Initializer/zeros', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/anton/.local/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/anton/.local/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/

All models was trained and reach almost the same accuracy ~ 0.9-1. Let's load one of them to plot the results.

In [8]:
config['load'] = {'path':'./DenseNet121/'}

In [9]:
%%time
model = DenseNet121(config=config)

INFO:tensorflow:Restoring parameters from ./DenseNet121/model-1383
CPU times: user 5min 31s, sys: 8.58 s, total: 5min 39s
Wall time: 5min 38s


Create pipeline to predict numbers on meters

In [10]:
predict_ppl = (
    (load_ppl << dset.test)
    .init_variable('prediction', init_on_each_run=list)
    .init_variable('proba', init_on_each_run=list)
    .init_variable('accuracy', init_on_each_run=list)
    .import_model('DenseNet121', model)
    .predict_model('DenseNet121',
                   fetches=['output_labels',
                            'output_proba',
                            'output_accuracy'],
                   make_data=make_separate_digits,
                   save_to=[V('prediction'),
                            V('proba'),
                            V('accuracy')],
                   mode='a')
)

Plot prediction numbers

In [12]:
batch_test = predict_ppl.next_batch(1, n_epochs=None, shuffle=True)

predict = predict_ppl.get_variable('prediction')[-1]
labels = batch_test.labels[-1]
sepcrop = batch_test.sepcrop[-1]
plt.title('Input image', fontsize=20)
plt.imshow(batch_test.images[-1])
plt.grid()
plt.show()

plt.title('Bounding box', fontsize=20)
plt.imshow(batch_test.cropped[-1])
plt.grid()

_, axis = plt.subplots(1, 8, figsize=(20, 10))
axis = axis.reshape(-1)
for i in range(8):
    axis[i].imshow(sepcrop[i])
    axis[i].set_title('Predict: {} \nAnswer: {}'.format(predict[i], labels[i]), fontsize=20)
    axis[i].grid()
    axis[i].axis('off')
plt.show()

Summing up in this example we learned:
* how to use meter batch
* categorize a number on water meter